# Downloading PDF-files

First it starts with preparations and then it goes through the Data Frame for each PDF-file in it.

In [1]:
# Here's where installations and imports from libraries are.

# This one at the top was mentioned in the original program, so I might use it if necessary.
#!pip install PyPDF2

import pandas as pd
#import os
import requests
#import time

In [2]:
# The excel files are read.

#meta_data = pd.read_excel("Metadata2006_2016.xlsx")
#GRI = pd.read_excel("C:/Users/KOM/Documents/GRI_2017_2020 (1).xlsx")
GRI = pd.read_excel("C:/Users/KOM/Documents/GRI_2017_2020 (1).xlsx")

In [3]:
# Then put into Data Frames, so they are easier to work with and looks more nice.

#dfm = pd.DataFrame(meta_data)
dfg = pd.DataFrame(GRI)

In [4]:
# So we can see the first 5 rows of GRI_2017_2020 (1).xlsx in a Data Frame.

#dfg.head(21034)

In [5]:
# To get further info about GRI_2017_2020 (1).xlsx, also a describtion. This was way worse when it wasn't a Data Frame.

#print("Info")
#print(dfg.info())
#print()
#print("Describtion")
#print(dfg.describe())
#print()
#print("Head")
#print(dfg.head())

In [6]:
# Just trying to get the ID's of GRI_2017_2020 (1).xlsx and then print it out. 
# This also shows it's not an integer, but a string/object.

#a = dfg.get('BRnum')
#print(a)

In [7]:
# Replacing the NaN with empty space.

dfg.fillna('', inplace=True)

#dfg.head(21034)

In [8]:
# This one would replace the empty spaces in the Pdf_URL column with a text.
#new_dfg['Pdf_URL'] = new_dfg['Pdf_URL'].replace([''],'Report Html Address'

# Replaces the empty space with a copy of the value of the column next to it in the same row.
#new_dfg.loc[new_dfg['Pdf_URL']=='', 'Pdf_URL'] = new_dfg['Report Html Address']
dfg.loc[dfg['Pdf_URL']=='', 'Pdf_URL'] = dfg['Report Html Address']

#new_dfg.head(21034)
#dfg.head(21034)

In [9]:
# Testing to see if the persumed value is accurate or not
text = 'https://www.awm-muenchen.de/fileadmin/PDF-Dokumente/awm/2016_AWM_Geschaeftsbericht.pdf'

assert text == dfg.loc[25,'Pdf_URL'], "It should be " + dfg.loc[25,'Pdf_URL']

In [10]:
# So we can see first 5 rows of the Data Frame of Metadata2006_2016.xlsx.

#dfm.head(2075)

In [11]:
# To get further info about Metadata2006_2016.xlsx, also a describtion.

#print("Info")
#print(dfm.info())
#print()
#print("Describtion")
#print(dfm.describe())
#print()
#print("Head")
#print(dfm.head())

In [12]:
# Replaces all the 'yes' in Metadata with 'Downloaded'.
#dfm['pdf_downloaded'] = dfm['pdf_downloaded'].str.replace('yes','Downloaded')

# Replaces all the 404, 'EOF marker not found', 'File has not been decrypted' and '[Errno 22] Invalid argument' with 'Not downloaded'.
#dfm['pdf_downloaded'] = dfm['pdf_downloaded'].replace([str(404),'EOF marker not found', 'File has not been decrypted', '[Errno 22] Invalid argument', 'Could not read malformed PDF file'], 'Not Dowloaded')

#dfm.head(2075)

In [13]:
# Testing to see if the persumed value is accurate or not
#text = 'Downloaded'

#assert text == dfm.loc[3,'pdf_downloaded'], "It should be " + dfm.loc[3,'pdf_downloaded']

In [14]:
# Making a connection so the PDF files are downloaded the right place on the computer.
output_dir = 'C:/Users/KOM/Documents/PDF_Files'

In [15]:
# Making a data frame with the Pdf_URLs and BRnum, but it tells if the file was downloaded or not.
check_GRI = pd.DataFrame()
check_GRI['BRnum'] = []
check_GRI['Pdf_URL'] = []
check_GRI['downloaded?'] = []

#print(check_GRI)

In [16]:
# This loop is where the magic happens.
for index, e in dfg.iterrows():
    if index < 50:
        try:
            # First it appends the BRnumber and link to the PDF from dfg to check_GRI
            check_GRI = check_GRI.append({'BRnum' : e['BRnum'], 'Pdf_URL' : e['Pdf_URL']}, ignore_index = True)
            # It gets a response from the URL from the PDF, and if the response works, is only 200, it ends with '.pdf', and the content type ends with 'pdf', the loop starts.
            # It's also set to a timeout if 5 seconds passes.
            response = requests.get(e['Pdf_URL'], timeout = 5)
            if (response.status_code == 200) and (e['Pdf_URL'].endswith('.pdf')) and (response.headers['Content-Type'].startswith('application/pd')):
                dst = output_dir + '/' + e['BRnum'] + '.pdf'
                # Here the content of the file is inserted and named as its BRnumber.
                with open(dst, 'wb') as f:
                    f.write(response.content)
                    # And changes the value in 'downloaded?' in the current index to 'Downloaded'.
                    check_GRI.loc[index,'downloaded?'] = 'Downloaded'
            # If the conditions are not met, the loop goes through the second column, and does almost the same as the one above.
            else:
                response = requests.get(e['Report Html Address'], timeout = 5)
                if (response.status_code == 200) and (e['Report Html Address'].endswith('.pdf')) and (response.headers['Content-Type'].startswith('application/pd')):
                    dst = output_dir + '/' + e['BRnum'] + 'r.pdf'
                    with open(dst, 'wb') as f:
                            f.write(response.content)
                            check_GRI.loc[index,'Pdf_URL'] = e['Report Html Address']
                            check_GRI.loc[index,'downloaded?'] = 'Downloaded'
                else: 
                    check_GRI.loc[index,'downloaded?'] = 'Not Downloaded'
        # If there comes an error in the reading, it'll usually either try with second column or the 'downloaded' will be 'Not Downloaded'
        except requests.exceptions.RequestException as error:
            #print("Error in " + e['BRnum'] + " : ", error)
            try:
                # It gets a response from the URL from the PDF, and if the response works, is only 200, it ends with '.pdf', and the content type ends with 'pdf', the loop starts.
                response = requests.get(e['Report Html Address'], timeout = 5)
                if (response.status_code == 200) and (e['Report Html Address'].endswith('.pdf')) and (response.headers['Content-Type'].startswith('application/pd')):
                    #If it meets the conditions, then the URL will be replaced by the value next to it.
                    check_GRI.loc[index,'Pdf_URL'] = e['Report Html Address']
                    dst = output_dir + '/' + e['BRnum'] + 'r.pdf'
                    # Here the content of the file is inserted and named as its BRnumber.
                    with open(dst, 'wb') as f:
                        f.write(response.content)
                        check_GRI.loc[index,'downloaded?'] = 'Downloaded'
                else:
                    check_GRI.loc[index,'downloaded?'] = 'Not Downloaded'
            except requests.exceptions.RequestException as error:
                check_GRI.loc[index,'downloaded?'] = 'Not Downloaded'
                #print("Error in " + e['BRnum'] + " : ", error)
            # When the time runs out on a Pdf file it stops there but is still downloaded.
            except requests.exceptions.ConnectTimeout:
                check_GRI.loc[index,'downloaded?'] = 'Downloaded'
        except requests.exceptions.ConnectTimeout:
            check_GRI.loc[index,'downloaded?'] = 'Downloaded'

C:\Users\KOM\AppData\Local\Temp\ipykernel_5424\845984795.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  check_GRI = check_GRI.append({'BRnum' : e['BRnum'], 'Pdf_URL' : e['Pdf_URL']}, ignore_index = True)
C:\Users\KOM\AppData\Local\Temp\ipykernel_5424\845984795.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  check_GRI = check_GRI.append({'BRnum' : e['BRnum'], 'Pdf_URL' : e['Pdf_URL']}, ignore_index = True)
C:\Users\KOM\AppData\Local\Temp\ipykernel_5424\845984795.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  check_GRI = check_GRI.append({'BRnum' : e['BRnum'], 'Pdf_URL' : e['Pdf_URL']}, ignore_index = True)
C:\Users\KOM\AppData\Local\Temp\ipykernel_5424\845984795.py:6: FutureWarning: The frame.append method is d

In [17]:
#check_GRI.fillna('Not Downloaded', inplace=True)

#print(check_GRI)

In [18]:
# Testing to see if the persumed value is accurate or not
text = 'Downloaded'

assert text == check_GRI.loc[19,'downloaded?'], "It should be " + check_GRI.loc[19,'downloaded?']

In [19]:
# Converts the Data Frame into an excel file.
check_GRI.to_excel('C:/Users/KOM/Documents/check_GRI.xlsx', index = False)